In [1]:
import matplotlib.pyplot as plt
from gerrychain import Partition, Graph, MarkovChain, Election, updaters, accept
from gerrychain.proposals import recom
from functools import partial
import json
import jsonlines as jl
import random
import click
import warnings

import pandas as pd
import geopandas as gpd
import numpy as np


from tqdm import tqdm

warnings.filterwarnings("ignore")


### Jefferson County dual graph:

In [2]:
graph = Graph.from_json('./Jefferson_County_precincts.json')

In [3]:
graph.nodes[0]

{'boundary_node': False,
 'area': 4630472.57580736,
 'STATEFP20': '08',
 'COUNTYFP20': '059',
 'VTDST20': '059040',
 'GEOID20': '08059059040',
 'VTDI20': 'A',
 'NAME20': 'Jefferson 040',
 'NAMELSAD20': 'Jefferson 040',
 'LSAD20': '00',
 'MTFCC20': 'G5240',
 'FUNCSTAT20': 'N',
 'ALAND20': 4601450,
 'AWATER20': 32622,
 'INTPTLAT20': '+39.6727134',
 'INTPTLON20': '-105.3508621',
 'tot_pop_20': 2129,
 'bpop_20': 9,
 'hpop_20': 48,
 'asian_nhpi_pop_20': 42,
 'amin_pop_20': 25,
 'other_pop_20': 19,
 'white_pop_20': 1986,
 'tot_vap_20': 1580,
 'bvap_20': 3,
 'hvap_20': 30,
 'asian_nhpi_vap_20': 25,
 'amin_vap_20': 11,
 'other_vap_20': 16,
 'white_vap_20': 1495,
 'pres_08_dem': 592,
 'pres_08_rep': 808,
 'pres_12_dem': 510,
 'pres_12_rep': 933,
 'pres_16_dem': 678,
 'pres_16_rep': 698,
 'sen_16_dem': 665,
 'sen_16_rep': 753,
 'gov_18_dem': 687,
 'gov_18_rep': 722,
 'ag_18_dem': 644,
 'ag_18_rep': 746,
 'pres_20_dem': 902,
 'pres_20_rep': 669,
 'sen_20_dem': 819,
 'sen_20_rep': 793}

### Add an NHVAP column:

In [4]:
for node in graph.nodes():
    graph.nodes[node]['nhvap_20'] = graph.nodes[node]['tot_vap_20'] - graph.nodes[node]['hvap_20']

In [5]:
graph.nodes[0]

{'boundary_node': False,
 'area': 4630472.57580736,
 'STATEFP20': '08',
 'COUNTYFP20': '059',
 'VTDST20': '059040',
 'GEOID20': '08059059040',
 'VTDI20': 'A',
 'NAME20': 'Jefferson 040',
 'NAMELSAD20': 'Jefferson 040',
 'LSAD20': '00',
 'MTFCC20': 'G5240',
 'FUNCSTAT20': 'N',
 'ALAND20': 4601450,
 'AWATER20': 32622,
 'INTPTLAT20': '+39.6727134',
 'INTPTLON20': '-105.3508621',
 'tot_pop_20': 2129,
 'bpop_20': 9,
 'hpop_20': 48,
 'asian_nhpi_pop_20': 42,
 'amin_pop_20': 25,
 'other_pop_20': 19,
 'white_pop_20': 1986,
 'tot_vap_20': 1580,
 'bvap_20': 3,
 'hvap_20': 30,
 'asian_nhpi_vap_20': 25,
 'amin_vap_20': 11,
 'other_vap_20': 16,
 'white_vap_20': 1495,
 'pres_08_dem': 592,
 'pres_08_rep': 808,
 'pres_12_dem': 510,
 'pres_12_rep': 933,
 'pres_16_dem': 678,
 'pres_16_rep': 698,
 'sen_16_dem': 665,
 'sen_16_rep': 753,
 'gov_18_dem': 687,
 'gov_18_rep': 722,
 'ag_18_dem': 644,
 'ag_18_rep': 746,
 'pres_20_dem': 902,
 'pres_20_rep': 669,
 'sen_20_dem': 819,
 'sen_20_rep': 793,
 'nhvap_20'

### We should tally all these population and election columns for districts.

In [6]:
election_names = [
    'pres_08', 'pres_12', 'pres_16', 'sen_16', 'gov_18', 'ag_18', 'pres_20', 'sen_20'
]

election_columns = [
    [col+'_dem', col+'_rep'] for col in election_names
]

# Add an "election" for hvap percentage:

election_names.append('hvap_20')
election_columns.append(['hvap_20', 'nhvap_20'])

num_elections = len(election_names)


elections = [
    Election(
        election_names[i],
        {"Democratic": election_columns[i][0], "Republican": election_columns[i][1]},
    )
    for i in range(num_elections)
]

In [7]:
my_updaters = {"totpop20": updaters.Tally('tot_pop_20', alias="totpop20")}

election_updaters = {election.name: election for election in elections}

my_updaters.update(election_updaters)

### Initial partitions for 3 and 5 district plans:

In [8]:
initial_partition_3 = Partition.from_random_assignment(
    graph,
    n_parts=3,
    epsilon=0.05,
    pop_col="tot_pop_20",
    updaters=my_updaters
)

initial_partition_5 = Partition.from_random_assignment(
    graph,
    n_parts=5,
    epsilon=0.05,
    pop_col="tot_pop_20",
    updaters=my_updaters
)


In [9]:
ideal_pop_3 = int(sum(initial_partition_3["totpop20"].values())/3)
ideal_pop_5 = int(sum(initial_partition_3["totpop20"].values())/5)

print(ideal_pop_3, ideal_pop_5)

194303 116582


In [10]:
proposal_3 = partial(
    recom, pop_col="tot_pop_20", pop_target=ideal_pop_3, epsilon=0.05
)

proposal_5 = partial(
    recom, pop_col="tot_pop_20", pop_target=ideal_pop_5, epsilon=0.05
)


recom_chain_3 = MarkovChain(
    proposal_3,
    constraints=[],
    accept=accept.always_accept,
    initial_state=initial_partition_3,
    total_steps=100000,
)

recom_chain_5 = MarkovChain(
    proposal_5,
    constraints=[],
    accept=accept.always_accept,
    initial_state=initial_partition_5,
    total_steps=100000,
)



### Collect stats:

In [11]:
chain_3_data_dict={}

chain_3_data_dict['totpop20']=[]
chain_3_data_dict['vap20']=[]
chain_3_data_dict['hvap20'] = []

for election in election_names:
    chain_3_data_dict[election]=[]


for part in tqdm(recom_chain_3):
        
    chain_3_data_dict['totpop20'].append(sorted(list(part["totpop20"].values())))
    
    for election in election_names:
        chain_3_data_dict[election].append(sorted(part[election].percents("Democratic")))


with(open('./Outputs/Chain_3_totpop20.json', 'w')) as pout:
    json.dump(chain_3_data_dict['totpop20'] , pout)  

with(open('./Outputs/Chain_3_vap20.json', 'w')) as pout:
    json.dump(chain_3_data_dict['vap20'] , pout)  

with(open('./Outputs/Chain_3_hvap20.json', 'w')) as pout:
    json.dump(chain_3_data_dict['hvap20'] , pout)  

            
for election in election_names:                
    with(open('./Outputs/Chain_3_' + election + '.json', 'w')) as eout:
        json.dump(chain_3_data_dict[election] , eout)  
                


100%|█████████████████████████████████| 100000/100000 [08:30<00:00, 196.03it/s]


In [13]:
chain_5_data_dict={}

chain_5_data_dict['totpop20']=[]
chain_5_data_dict['vap20']=[]
chain_5_data_dict['hvap20'] = []

for election in election_names:
    chain_5_data_dict[election]=[]


for part in tqdm(recom_chain_5):
        
    chain_5_data_dict['totpop20'].append(sorted(list(part["totpop20"].values())))
    
    for election in election_names:
        chain_5_data_dict[election].append(sorted(part[election].percents("Democratic")))


with(open('./Outputs/Chain_5_totpop20.json', 'w')) as pout:
    json.dump(chain_5_data_dict['totpop20'] , pout)  

with(open('./Outputs/Chain_5_vap20.json', 'w')) as pout:
    json.dump(chain_5_data_dict['vap20'] , pout)  

with(open('./Outputs/Chain_5_hvap20.json', 'w')) as pout:
    json.dump(chain_5_data_dict['hvap20'] , pout)  

            
for election in election_names:                
    with(open('./Outputs/Chain_5_' + election + '.json', 'w')) as eout:
        json.dump(chain_5_data_dict[election] , eout)  
                


100%|█████████████████████████████████| 100000/100000 [06:11<00:00, 268.85it/s]
